In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

# NLP libraries
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

# ML libraries
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# Neural Networks libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.layers import Dense, LSTM, Dropout, Bidirectional,Flatten
from keras.models import Sequential

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# lemmatizer
lm = WordNetLemmatizer()

# Checks if given word contains a special character
def contains_special(word):
    for char in word:
        if char.isnumeric() or (not char.isalnum()):
            return True
    return False

# process sentences
def process_sentences(sentence):
    # tokenize, lemmatize, and remove special characters
    processed = [lm.lemmatize(word.lower()) for word in nltk.word_tokenize(sentence)
          # make sure no strings that contain only numeric characters
          if not contains_special(word)]
    return ' '.join(processed)

In [3]:
# get data
df = pd.read_csv('tweets.csv')
# drop duplicates
df.drop_duplicates(inplace =True)
X = df['text']
y = df['target']

In [4]:
y.value_counts()

0    9256
1    2114
Name: target, dtype: int64

In [5]:
# text pre-processing
X = X.apply(process_sentences)

In [6]:
X[0]

'communal violence in bhainsa telangana stone were pelted on muslim house and some house and vehicle were set'

In [7]:
# Convert text data into BoW features
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)
vocabulary = vectorizer.vocabulary_
print('Vocab size: ' + str(len(vocabulary)))

Vocab size: 16985


In [9]:
vocabulary['house']

6955

In [ ]:
# split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), y, test_size=0.25, random_state=1)

In [ ]:
# Naive Bayes
def naive_bayes(X_train, X_test, y_train, y_test):
    # increased max iter to prevent early time out
    clf = MultinomialNB().fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('accuracy:', metrics.accuracy_score(y_test, y_pred))
    print(metrics.classification_report(y_test,
                                    y_pred))
    return y_pred

print('Naive Bayes\n')
ypred_lr = naive_bayes(X_train, X_test, y_train, y_test)

In [4]:
# Logistic Regression
def logistic_regression(X_train, X_test, y_train, y_test):
    scaler = StandardScaler()
    # scale data to reduce ranges
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    # increased max iter to prevent early time out
    clf = LogisticRegression(max_iter = 500).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('accuracy:', metrics.accuracy_score(y_test, y_pred))
    print(metrics.classification_report(y_test,
                                    y_pred))
    return y_pred

print('Logistic Regression\n')
ypred_lr = logistic_regression(X_train, X_test, y_train, y_test)

Logistic Regression



NameError: ignored

In [10]:
# get data
df = pd.read_csv('tweets.csv')
# drop duplicates
df.drop_duplicates(inplace =True)

X = df['text']
y = df['target']

# text pre-pro|cessing
X = X.apply(process_sentences)

In [55]:
from gensim.models import Word2Vec

def get_embeddings(data, size):
  return Word2Vec(sentences=data, vector_size=size, window=5, min_count=1,
                 sg=1)

# get a mapping of word -> embedding, word_index -> embedding
def map_embeddings(model, tokenizer):
    # all vocab
    vocab = list(model.wv.index_to_key)
    # word to word index map
    word_index = tokenizer.word_index
    # word to embedding map
    word_embed = {}
    # word index to embedding map
    index_embed = {}

    for w in vocab:
        # get current word's embedding
        e = model.wv[w]
        # set embedding for current word
        word_embed[w] = e
        # set embedding for current word index
        curr_index = word_index[w]
        index_embed[curr_index] = e
    return word_embed, index_embed

# pre-padding data to fixed length token for NN input
def pre_padding(encoded, seq_length, model):
    if model == "rnn":
      X = []
      for row in encoded:
          for i in range(1, len(row) - 1):
              X.append(row[:i])
    X = pad_sequences(encoded, maxlen = seq_length)
    return X

# convert data into 3D matrix (total rows, max sequence length, embedding size) for LSTM
def reshape_data(X, index_embed):
  reshaped = []
  for seq in X:
    # get embeddings for each word index in sequence
    seq_embed = [index_embed[index] for index in seq]
    reshaped.append(seq_embed)
  # convert to numpy array
  return np.array(reshaped)


def data_generator(X, y, num_sequences, index_embed, model) -> (list,list):
    i = 0
    while i < len(X):
        # end range of data for current batch
        end_index = i + num_sequences
        # reached end of our dataset
        if end_index >= len(X) - 1:
            i = 0
            end_index = i + num_sequences

        if model == "ffnn":
          # we need to flatten our inputs for a feedforward network
          inputs = [input.flatten() for input in X[i:end_index]]
        else:
          inputs = [val for val in X[i:end_index]]

        outputs = [val for val in y[i:end_index]]
        yield np.array(inputs), np.array(outputs)
        i += num_sequences


In [12]:
# process data for Gensim's Word2Vec
X = X.apply(nltk.word_tokenize)
word2vec = get_embeddings(X,200)
vocab_size = len(word2vec.wv.index_to_key)
print('Vocab size {}'.format(vocab_size))

Vocab size 17020


In [13]:
word2vec.wv.most_similar('people', topn = 10)

[('those', 0.9485560059547424),
 ('these', 0.9232138395309448),
 ('their', 0.905163586139679),
 ('many', 0.8945909738540649),
 ('american', 0.8919721841812134),
 ('innocent', 0.886820375919342),
 ('who', 0.8828123211860657),
 ('asking', 0.8800298571586609),
 ('other', 0.8794156312942505),
 ('animal', 0.8793823719024658)]

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
embedding_size = 200
# embeddings
word_embed, index_embed = map_embeddings(word2vec, tokenizer)
# set embeddings for zero index
index_embed[0] = np.zeros((embedding_size,))
word_embed[''] = np.zeros((embedding_size,))
vocab_size = len(word_embed.keys())

In [15]:
# encode data into word indexes
encoded_data = tokenizer.texts_to_sequences(X)
# max sequence length
max_sequence_length = np.max([len(seq) for seq in encoded_data])
# prepadding on data
ffnn_pre_padded_X = pre_padding(encoded_data, max_sequence_length, "ffnn")
rnn_pre_padded_X = pre_padding(encoded_data, max_sequence_length, "rnn")

In [26]:
# convert pre-padded and encoded data into 3D shape for FFNN input
ffnn_X = reshape_data(ffnn_pre_padded_X, index_embed)
print(ffnn_X.shape)

(11370, 30, 200)


In [17]:
# convert pre-padded and encoded data into 3D shape for FFNN input
rnn_X = reshape_data(rnn_pre_padded_X, index_embed)
print(rnn_X.shape)

(11370, 30, 200)


In [49]:
X_train, X_test, y_train, y_test = train_test_split(ffnn_X, y, test_size=0.25, random_state=1)

In [50]:
# Feedforward Neural Network
# hyperparameters
epochs = 10
batches = 256
steps = len(X_train)//batches

# data generator
train_generator = data_generator(X_train, y_train, batches, index_embed, "ffnn")

# architecture
model = Sequential()
model.add(Dense(200, activation='relu', input_dim = max_sequence_length * embedding_size))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
# sigmoid for binary classification
model.add(Dense(units= 1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x= train_generator,
          steps_per_epoch= steps,
          epochs= epochs, verbose = 1)

print(model.output)

Epoch 1/10
33/33 [==============================] - 3s 47ms/step - loss: 0.4298 - accuracy: 0.8162
Epoch 2/10
33/33 [==============================] - 2s 47ms/step - loss: 0.3635 - accuracy: 0.8499
Epoch 3/10
33/33 [==============================] - 2s 46ms/step - loss: 0.3460 - accuracy: 0.8581
Epoch 4/10
33/33 [==============================] - 2s 47ms/step - loss: 0.3354 - accuracy: 0.8614
Epoch 5/10
33/33 [==============================] - 3s 83ms/step - loss: 0.3249 - accuracy: 0.8691
Epoch 6/10
33/33 [==============================] - 3s 76ms/step - loss: 0.3171 - accuracy: 0.8714
Epoch 7/10
33/33 [==============================] - 2s 71ms/step - loss: 0.3128 - accuracy: 0.8717
Epoch 8/10
33/33 [==============================] - 2s 56ms/step - loss: 0.3091 - accuracy: 0.8751
Epoch 9/10
33/33 [==============================] - 2s 47ms/step - loss: 0.2997 - accuracy: 0.8808
Epoch 10/10
33/33 [==============================] - 2s 46ms/step - loss: 0.2906 - accuracy: 0.8833
KerasTens

In [53]:
X_train, X_test, y_train, y_test = train_test_split(rnn_X, y, test_size=0.25, random_state=1)

In [56]:
# LSTM
# hyperparameters
epochs = 5
batches = 256
steps = len(X_train)//batches

# Data generator
train_generator = data_generator(X_train, y_train, batches, index_embed, "rnn")

# Architecture
model = Sequential()
# LSTM input layer
model.add(LSTM(200, input_shape=(max_sequence_length, embedding_size),return_sequences=True))
# Dropout layer to prevent overfitting
model.add(Dropout(0.2))
# LSTM layer
model.add(LSTM(128, input_shape=(max_sequence_length, embedding_size),return_sequences=True))
model.add(LSTM(32))
model.add(Dropout(0.2))
# sigmoid for binary classification
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x= train_generator,
          steps_per_epoch= steps,
          epochs= epochs, verbose = 1)

print(model.output)

Epoch 1/5
33/33 [==============================] - 29s 691ms/step - loss: 0.4355 - accuracy: 0.8252
Epoch 2/5
33/33 [==============================] - 20s 623ms/step - loss: 0.3662 - accuracy: 0.8475
Epoch 3/5
33/33 [==============================] - 21s 648ms/step - loss: 0.3566 - accuracy: 0.8513
Epoch 4/5
33/33 [==============================] - 22s 642ms/step - loss: 0.3496 - accuracy: 0.8546
Epoch 5/5
33/33 [==============================] - 21s 624ms/step - loss: 0.3458 - accuracy: 0.8544
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='dense_42/Sigmoid:0', description="created by layer 'dense_42'")


In [57]:
# Recurrent Neural Networks : Bidirectional LSTM
# hyperparameters
epochs = 5
batches = 256
steps = len(X_train)//batches

# Data generator
train_generator = data_generator(X_train, y_train, batches, index_embed, "rnn")

# Model architecture
model = Sequential()
# LSTM input layer
model.add(LSTM(128, input_shape=(max_sequence_length, embedding_size),return_sequences=True))
model.add(LSTM(64, input_shape=(max_sequence_length, embedding_size),return_sequences=True))
# Dropout layer to prevent overfitting
model.add(Dropout(0.2))
# Bidirectional LSTM layer for extra context
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.2))
# sigmoid for binary classification
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x= train_generator,
          steps_per_epoch= steps,
          epochs= epochs, verbose = 1)

print(model.output)

Epoch 1/5
33/33 [==============================] - 21s 362ms/step - loss: 0.4967 - accuracy: 0.8037
Epoch 2/5
33/33 [==============================] - 13s 395ms/step - loss: 0.3864 - accuracy: 0.8365
Epoch 3/5
33/33 [==============================] - 13s 404ms/step - loss: 0.3629 - accuracy: 0.8519
Epoch 4/5
33/33 [==============================] - 14s 415ms/step - loss: 0.3528 - accuracy: 0.8536
Epoch 5/5
33/33 [==============================] - 21s 636ms/step - loss: 0.3461 - accuracy: 0.8570
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='dense_43/Sigmoid:0', description="created by layer 'dense_43'")
